<a href="https://colab.research.google.com/github/psychopatz/Colab-Torrent-Downloader-RELOADED/blob/main/Colab_Torrent_Downloader_Reloaded.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://bit.ly/3xlijDv" target="_blank">
<img src="https://icons.iconarchive.com/icons/neiio/prime-dock-2/256/uTorrent-icon.png" height="100px" align="left"></a>

# <font size=6><center>**Torrent Download <font color=#e53c1d>Re</font><font color=#16bf2d>loa</font><font color=#158ff5>ded</font>**</center></font>



A feature-rich **Torrent Download manager** powered by Google Collab.

**Features:**

*Built-in Torrent Search engine for fast secure search.

*History and resume downloads support.

*VPN-like features both downloading and searching that prevents DMCA Notice since it technically uses Google's internet instead of yours.


Visit https://bit.ly/3xlijDv for more info.


In [7]:
#@title Initialize Downloader

!python -m pip install --upgrade pip setuptools wheel
!python -m pip install lbry-libtorrent
!apt install python3-libtorrent
!pip install bencodepy
!pip install tpblite



import os
import sys
import time
import shutil
import base64
import hashlib
import bencodepy
import libtorrent as lt
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output
from google.colab import output
from google.colab import files
from tpblite import TPB, CATEGORIES

time.sleep(2)
clear_output()
print('Importing Done..Now Authenticating GDrive ,')

#Drive Authentication
from google.colab import drive
drive.mount("/content/drive")


#Environmental Variables
TorrentHistory = '/content/drive/My Drive/Torrent/TorrentHistory/'
SaveDirectory = "/content/drive/My Drive/Torrent" #@param {type:"string"}
completed_path = TorrentHistory + 'Completed/'

params = {"save_path": SaveDirectory}

for path in [TorrentHistory, SaveDirectory, completed_path]:
 if not os.path.exists(path):
	  os.makedirs(path)

#Initialize
ses = lt.session()
ses.listen_on(6881, 6891)
downloads = []

time.sleep(1)
clear_output()

print ('Initialize Required Functions.')
# extract magnet from .torrent
def torrent2Magnet(file) :
    metadata = bencodepy.decode_from_file(file)
    subj = metadata[b'info']
    hashcontents = bencodepy.encode(subj)
    digest = hashlib.sha1(hashcontents).digest()
    b32hash = base64.b32encode(digest).decode()
    return 'magnet:?'\
             + 'xt=urn:btih:' + b32hash\
             + '&dn=' + metadata[b'info'][b'name'].decode()\
             + '&tr=' + metadata[b'announce'].decode()\
             + '&xl=' + str(metadata[b'info'][b'piece length'])

time.sleep(1)
clear_output()
print ('Initialize Required Functions..')
#Verify the magnet and download its metadata
def magnet2Torrent(magnet_link,TorrentHistory):
    try:
        params = lt.parse_magnet_uri(magnet_link)
    except:
        print("Error: Only valid Magnet link is accepted!")
        return False

    session = lt.session()
    handle = session.add_torrent(params)
    timer = 0
    print("Downloading Metadata, Please Wait...")
    while not handle.has_metadata(): 
        timer = timer + 1
        time.sleep(0.1)
        if timer >= 1000: #1 min estimation
            print("Timeout Error, try next time or use other magnet.")
            return False 
        

# Create .torrent and save to the history
    torrent_info = handle.get_torrent_info()
    torrent_file = lt.create_torrent(torrent_info)
    torrent_path = os.path.join(TorrentHistory, torrent_info.name() + ".torrent")
    
    with open(torrent_path, "wb") as f:
        f.write(lt.bencode(torrent_file.generate()))
    print ("%s is Added" % torrent_info.name())
    return True

#Show Queue
def showQueue():
  if downloads:
    print ("\nDownload Queue:")
    for index, download in enumerate(downloads[:]):
        print ((index + 1),".", download.name())

#Scan for .torrents
def showUnfinished():
  paths = os.listdir(TorrentHistory)
  return [path for path in paths if path.endswith('.torrent')]
 
#Transfer .torrent when completed 
def transferTorrent(torrentName):
  try:
    shutil.move(TorrentHistory + torrentName + '.torrent', completed_path + torrentName + '.torrent')
  except:
    return



#Load All .torrent
def resumeTorrent():
    #Scan for .torrents
    paths = showUnfinished()

    #Add .torrent
    for data in paths:
        magnet_link = torrent2Magnet(TorrentHistory + data)
        #print (magnet_link)
        downloads.append(lt.add_magnet_uri(ses, magnet_link, params))




#add Magnet Downloads
def addViaMagnet():
    while True:
        showQueue()
        magnet_link = input("Enter Magnet Link Or Type Exit: ")
        clear_output()
        if magnet_link.lower() == "exit":
            break
        if magnet2Torrent(magnet_link,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))

#add .torrent Downloads
def addViaTorrent():
    while True:
        showQueue()
        source = files.upload()
        filename = next(iter(source))
        magnet_link = torrent2Magnet(filename)
        if magnet2Torrent(magnet_link,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnet_link, params))
        del source
        clear_output()

time.sleep(1)
clear_output()
print ('Initialize Required Functions...')

print('Importing Done..Mounting GDrive Done,')
print('Initializiation Complete,')
time.sleep(1)
print('Now you can run the next cell')



Initialize Required Functions...
Importing Done..Mounting GDrive Done,
Initializiation Complete,
Now you can run the next cell


In [8]:
#@title Use the Program




#Main
params = {"save_path": SaveDirectory}
Menu = "Add.Magnet" #@param ["Add.Magnet", "Add.torrent", "ResumeDownloads"]


if Menu == "Add.Magnet" :
    addViaMagnet()
elif Menu == "Add.torrent" :
    try: 
        addViaTorrent()
    except:
      print ("Selection Canceled..")
elif Menu == "ResumeDownloads" :
    resumeTorrent()
    showQueue()


#elif Add == "resume downloads" planned feature



time.sleep(2)
clear_output()
print ("Now Downloading: \n")

state_str = [
    "Queued:",
    "Checking:",
    "Downloading Metadata:",
    "Downloading:",
    "Finished:",
    "Seeding",
    "Allocating:",
    "Checking fastresume:",
]

layout = widgets.Layout(width="auto")
style = {"description_width": "initial"}
download_bars = [
    widgets.FloatSlider(
        step=0.01, disabled=True, layout=layout, style=style
    )
    for _ in downloads
]
display(*download_bars)

while downloads:
    next_shift = 0
    for index, download in enumerate(downloads[:]):
        bar = download_bars[index + next_shift]
        if not download.is_seed():
            s = download.status()

            bar.description = " ".join(
                [
                    state_str[s.state],
                    download.name(),
                    ", Seeders: ",str(s.num_seeds),
                    ", Speed: ", str(s.download_rate / 1000),"kB/s",
                    
                ]
            )
            bar.value = s.progress * 100  
        else:
            next_shift -= 1
            transferTorrent(download.name())
            ses.remove_torrent(download)
            downloads.remove(download)
            bar.close() 
            download_bars.remove(bar)
            print(download.name(), "complete")
    time.sleep(1)

print("Program Terminated, No more pending downloads!")


Now Downloading: 



FloatSlider(value=0.0, disabled=True, layout=Layout(width='auto'), step=0.01, style=SliderStyle(description_wi…

[FTUApps.com] - NextUp TextAloud v4.0.62 Portable complete
Program Terminated, No more pending downloads!


In [ ]:
#@title Search Torrents

def printContents(torrents):
    index = 0
    for torrent in torrents:
        index = index + 1
        print("[%d] %s" % (index,torrent))

def detailedItemView(torrent):
    print("Selected torrent:\n")
    print("Name: " + torrent.title)
    print("Category: " + torrent.category)
    print("Size: " + torrent.filesize)
    print("Seeder: %d\n" % (torrent.seeds))
    print("Date Uploaded: " + torrent.upload_date)
    print("Uploader: " + torrent.uploader)
    print("VIP: %s" %(torrent.is_vip))
    print("Trusted: %s" %  (torrent.is_trusted))
    print("URL: " + torrent.url)

def askToDownload(magnetlink):
    option = input("Download File? Y/N ")
    while option.lower() not in ("y","n"):
        option = input("Invalid Syntax, Download File? Y/N: ")
    if option == "y":
        if magnet2Torrent(magnetlink,TorrentHistory):
            downloads.append(lt.add_magnet_uri(ses, magnetlink, params))
        showQueue()
        return True
    return False

Search = "Tutorial" #@param {type:"string"}
t = TPB()
torrents = t.search(Search)

while True:
    clear_output()
    print('There were {0} torrents in PirateBay found.\n'.format(len(torrents)))
    printContents(torrents)
    try:
        selectedIndex = int(input("Choose: "))
    except:
        print("Error: Only integers are allowed.")
        input("Press Enter to continue...")
        continue
    
    if selectedIndex <= 0 or selectedIndex > len(torrents):
        print("Invalid Input: Must be between 0-%d" %(len(torrents)))
        input("Press Enter to continue...")
        continue

    clear_output()
    torrent = torrents[selectedIndex-1]
    detailedItemView(torrent)

    if askToDownload(torrent.magnetlink):
        print ("\nImport Complete, Run the cell again to search.")    
        break
    
    

    





